In [2]:
%pip install --upgrade --quiet python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade --quiet  langchain langchain-neo4j langchain-openai langchain-experimental neo4j python-dotenv

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = AzureChatOpenAI(temperature=0, model_name="AI-KMS-4o")

custom_prompt_template = """
Sei un sistema di estrazione di conoscenza. 
Leggi il seguente testo e individua:
1) Le entità principali (concetti, tecniche, organizzazioni, persone, ecc.) 
2) Corrispondente testo associato a ciascuna entità
3) Le relazioni tra le entità, con un tipo relazionale appropriato (es: "utilizza", "richiede", "comprende", "prevede", ecc.)
4) Per ciascuna relazione, fornisci un breve snippet che spieghi perché esiste questa relazione

Respondi in un unico oggetto JSON con questa struttura:

{{
  "nodes": [
    {{
      "id": "<nome_entita>",
      "type": "<tipo_entita>",
      "properties": {{
         "snippet": "<testo_assocciato>"
      }}
    }}
    ...
  ],
  "relationships": [
    {{
      "source": "<nome_entita_sorgente>",
      "target": "<nome_entita_destinazione>",
      "type": "<tipo_relazione>",
      "properties": {{
        "snippet": "<testo_spiegazione_relazione>"
      }}
    }}
    ...
  ]
}}

Assicurati di:
- Non inserire testi fuori contesto.
- Mantenere il campo "type" su nodi e relazioni al valore più aderente al contenuto.
- Non aggiungere ulteriori chiavi rispetto allo schema JSON specificato.

Testo da analizzare:
{input}
"""

custom_prompt = PromptTemplate(
    template=custom_prompt_template,
    input_variables=["input"]
)

llm_transformer = LLMGraphTransformer(llm=llm)#, node_properties=True, relationship_properties=True)#, additional_instructions="Try to classify correctly the relationship type, also extract the relevant text snippet and return it as a property")#, prompt=custom_prompt)

In [4]:
import pdfplumber

file_path = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Leggiamo il PDF
pdf_text = extract_text_from_pdf(file_path)

# Stampiamo un'anteprima
print(pdf_text[:500])  # Stampa i primi 500 caratteri


Introduzione
Io sono il grande Sirius Cosmo, lo chef stellare per eccellenza. Se non mi conoscete ancora, preparatevi:
il mio nome è sinonimo di cucina galattica. In questo manuale vi insegnerò tutto quello che serve per
diventare veri cuochi, da Alpha Centauri fino alla Nebulosa del Granchio (dove, a proposito, non hanno
neanche un crostaceo decente). Impareremo insieme le licenze e le abilità fondamentali per cucinare
nello spazio senza mandare in tilt lʼintero sistema di supporto vitale della


In [5]:
import re

chapters = re.findall(r"Capitolo \d+", pdf_text)
chapter_3 = pdf_text[pdf_text.find("Capitolo 3") : pdf_text.find("Capitolo 4")]
len(chapter_3)


14282

In [6]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=chapter_3)]
graph_documents_bis = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents_bis[0].nodes}")
print(f"Relationships:{graph_documents_bis[0].relationships}")

Nodes:[Node(id='Tecniche Di Preparazione', type='Concept', properties={}), Node(id='Marinatura', type='Concept', properties={}), Node(id='Affumicatura', type='Concept', properties={}), Node(id='Fermentazione', type='Concept', properties={}), Node(id='Tecniche Di Impasto', type='Concept', properties={}), Node(id='Surgelamento', type='Concept', properties={}), Node(id='Marinatura A Infusione Gravitazionale', type='Technique', properties={}), Node(id='Marinatura Temporale Sincronizzata', type='Technique', properties={}), Node(id='Marinatura Psionica', type='Technique', properties={}), Node(id="Marinatura Tramite Reazioni D'Antimateria Diluite", type='Technique', properties={}), Node(id='Marinatura Sotto Zero A Polarità Inversa', type='Technique', properties={}), Node(id='Affumicatura A Stratificazione Quantica', type='Technique', properties={}), Node(id='Affumicatura Temporale Risonante', type='Technique', properties={}), Node(id='Affumicatura Psionica Sensoriale', type='Technique', prope

In [8]:
graph_documents_bis[0].relationships

[Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Marinatura', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Affumicatura', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Fermentazione', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Tecniche Di Impasto', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Surgelamento', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Marinatura', type='Concept', properties

In [18]:
graph_documents[0].relationships

[Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Marinatura', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Affumicatura', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Fermentazione', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Tecniche Di Impasto', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Tecniche Di Preparazione', type='Concept', properties={}), target=Node(id='Surgelamento', type='Concept', properties={}), type='INCLUDES', properties={}),
 Relationship(source=Node(id='Marinatura', type='Concept', properties

In [19]:
help(llm_transformer.convert_to_graph_documents)

Help on method convert_to_graph_documents in module langchain_experimental.graph_transformers.llm:

convert_to_graph_documents(documents: Sequence[langchain_core.documents.base.Document], config: Optional[langchain_core.runnables.config.RunnableConfig] = None) -> List[langchain_community.graphs.graph_document.GraphDocument] method of langchain_experimental.graph_transformers.llm.LLMGraphTransformer instance
    Convert a sequence of documents into graph documents.
    
    Args:
        documents (Sequence[Document]): The original documents.
        kwargs: Additional keyword arguments.
    
    Returns:
        Sequence[GraphDocument]: The transformed documents as graphs.



In [20]:
help(llm_transformer)

Help on LLMGraphTransformer in module langchain_experimental.graph_transformers.llm object:

class LLMGraphTransformer(builtins.object)
 |  LLMGraphTransformer(llm: langchain_core.language_models.base.BaseLanguageModel, allowed_nodes: List[str] = [], allowed_relationships: Union[List[str], List[Tuple[str, str, str]]] = [], prompt: Optional[langchain_core.prompts.chat.ChatPromptTemplate] = None, strict_mode: bool = True, node_properties: Union[bool, List[str]] = False, relationship_properties: Union[bool, List[str]] = False, ignore_tool_usage: bool = False, additional_instructions: str = '') -> None
 |  
 |  Transform documents into graph-based documents using a LLM.
 |  
 |  It allows specifying constraints on the types of nodes and relationships to include
 |  in the output graph. The class supports extracting properties for both nodes and
 |  relationships.
 |  
 |  Args:
 |      llm (BaseLanguageModel): An instance of a language model supporting structured
 |        output.
 |      

In [22]:

%pip install llama-index-embeddings-azure-openai llama-index-llms-azure-openai
%pip install llama-index


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: llama-index-embeddings-azure-openai in c:\users\smancone\onedrive - business integration partners spa\desktop\progetti\2402_prysmian\.conda\lib\site-packages (0.3.0)
  Using cached openai-1.61.1-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.61.1-py3-none-any.whl (463 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.51.2
    Uninstalling openai-1.51.2:
      Successfully uninstalled openai-1.51.2



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



   ---------------------------------------- 0.0/253.0 kB ? eta -:--:--
   ---------------------------------------- 253.0/253.0 kB 7.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/300.7 kB ? eta -:--:--
   --------------------------------------- 300.7/300.7 kB 19.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.4 kB ? eta -:--:--
   ---------------------------------------- 166.4/166.4 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.0.1
    Uninstalling pypdf-5.0.1:
      Successfully uninstalled pypdf-5.0.1
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.2.2
    Uninstalling certifi-2024.2.2:
      Successfully uninstalled certifi-2024.2.2


In [44]:

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.llms import ChatMessage

import logging
import sys

api_key = os.getenv("OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("OPENAI_API_VERSION")

llm = AzureOpenAI(
    model="gpt-4o",
    deployment_name="AI-KMS-4o",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

Settings.llm = llm
Settings.chunk_size = 512



In [45]:
#llm.chat({"type":"user", "input_value":"What is the capital of France?"})
#Error: Input should be a valid list [type=list_type, input_value={'type': 'user', 'input_v...the capital of France?'}, input_type=dict]
# correct input
response = llm.chat([
    ChatMessage(role="user", content="What is the capital of France?")
])

In [46]:
response

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The capital of France is **Paris**.')]), raw=ChatCompletion(id='chatcmpl-AzMcXtYQTnfDU7Fl98yxdAhng1NMv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is **Paris**.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1739188325, model='gpt-4o-2024-11-20', object='chat.completion', service_tier=None, system_fingerprint='fp_f3927aa00d', usage=CompletionUsage(completion_toke

In [26]:

from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display

In [28]:
help(SimpleDirectoryReader.load_data)

Help on function load_data in module llama_index.core.readers.file.base:

load_data(self, show_progress: 'bool' = False, num_workers: 'int | None' = None, fs: 'fsspec.AbstractFileSystem | None' = None) -> 'list[Document]'
    Load data from the input directory.
    
    Args:
        show_progress (bool): Whether to show tqdm progress bars. Defaults to False.
        num_workers  (Optional[int]): Number of workers to parallelize data-loading over.
        fs (Optional[fsspec.AbstractFileSystem]): File system to use. If fs was specified
            in the constructor, it will override the fs parameter here.
    
    Returns:
        List[Document]: A list of documents.



In [32]:
docs = SimpleDirectoryReader("../Hackapizza Dataset/Misc/").load_data()

In [33]:
docs

[Document(id_='b5392819-b26c-49fb-a3f7-7505c388c467', embedding=None, metadata={'file_path': 'c:\\Users\\smancone\\OneDrive - BUSINESS INTEGRATION PARTNERS SPA\\Desktop\\kaggle\\letspizza\\Hackapizza Dataset\\Misc\\dish_mapping.json', 'file_name': 'dish_mapping.json', 'file_type': 'application/json', 'file_size': 12993, 'creation_date': '2025-02-09', 'last_modified_date': '2025-02-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='{\r\n    "Alternate Realities Risotto": 0,\r\n    "Antipasto Celestiale": 1,\r\n    "Antipasto Stellare dell\'Eterna Armonia": 2,\r\n    "Armonia Cosmica alla Tavola d\'Oro": 3,\r\n    "Armonia Cosmica della F

In [47]:
from llama_index.core import StorageContext

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    docs,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
)

In [49]:
dir(index)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_index_from_nodes',
 '_callback_manager',
 '_delete_node',
 '_docstore',
 '_embed_model',
 '_extract_triplets',
 '_graph_store',
 '_index_struct',
 '_insert',
 '_is_protocol',
 '_kg_triplet_extract_fn',
 '_llm',
 '_llm_extract_triplets',
 '_max_object_length',
 '_object_map',
 '_parse_triplet_response',
 '_show_progress',
 '_storage_context',
 '_transformations',
 '_vector_store',
 'add_node',
 'as_chat_engine',
 'as_query_engine',
 'as_retriever',
 'build_index_from_n

In [57]:
index.graph_store.to_dict()

{'graph_dict': {'Alternate realities risotto': [['Has id', '0']],
  'Cosmofantasia di terra e mare nel vortice delle stelle': [['Has id', '17']],
  'Eclissi armonica di sapori': [['Has id', '34']],
  "Fenice sull'orizzonte degli eventi": [['Has id', '50']],
  'Galassia di fusilli sferici alla risonanza crononica': [['Has id', '67']],
  'Galassia nel piatto: sinfonia universale di aromi e sapori': [['Has id',
    '83']],
  'Il risveglio del drago celeste': [['Has id', '96']],
  'La balena sputafuoco': [['Has id', '112']],
  'Nebulosa celestiale di sogni quantici': [['Has id', '128']],
  'Odissea celestiale': [['Has id', '143']],
  'Pizza gio': [['Has id', '160']],
  'Pizza luca': [['Has id', '161']],
  'Portale di sapori arcani': [['Has id', '175']],
  'Sassi e sassolini': [['Has id', '190']],
  "Sinfonia cosmica all'alba di fenice": [['Has id', '205']],
  'Sinfonia cosmica ma fatta bene': [['Has id', '220']],
  'Sinfonia interstellare di fusilli del vento con nettare di sirena': [['Has